#### Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,IsolationForest
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

d:\MLProject\env_v1\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
d:\MLProject\env_v1\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_pass

#### Feature Engineering

In [2]:
data = pd.read_csv(f'..\data\Performance_data.csv')

In [3]:
data['total_score'] = data['math score'] + data['reading score'] + data['writing score']
data['pass_math'] = data['math score'] >= 60
data['pass_reading'] = data['reading score'] >= 60
data['pass_writing'] = data['writing score'] >= 60
data['pass_overall'] = data['pass_math'] & data['pass_reading'] & data['pass_writing']

In [4]:
upper_limit_math = data['math score'].mean() + 3*data['math score'].std()
lower_limit_math = data['math score'].mean() - 3*data['math score'].std()
upper_limit_reading = data['reading score'].mean() + 3*data['reading score'].std()
lower_limit_reading = data['reading score'].mean() - 3*data['reading score'].std()
upper_limit_writing = data['writing score'].mean() + 3*data['writing score'].std()
lower_limit_writing = data['writing score'].mean() - 3*data['writing score'].std()


data['math score'] = data['math score'].apply(lambda x: upper_limit_math if x > upper_limit_math else x)
data['math score'] = data['math score'].apply(lambda x: lower_limit_math if x < lower_limit_math else x)
data['reading score'] = data['reading score'].apply(lambda x: upper_limit_reading if x > upper_limit_reading else x)
data['reading score'] = data['reading score'].apply(lambda x: lower_limit_reading if x < lower_limit_reading else x)
data['writing score'] = data['writing score'].apply(lambda x: upper_limit_writing if x > upper_limit_writing else x)
data['writing score'] = data['writing score'].apply(lambda x: lower_limit_writing if x < lower_limit_writing else x)

##### Isolation Forest

In [ ]:
# Select the numerical columns for outlier removal
numerical_columns = ['math score', 'reading score', 'writing score', 'average_score']

# Initialize the Isolation Forest model
isolation_forest = IsolationForest(contamination='auto', random_state=42)

# Fit the Isolation Forest model on the numerical columns
isolation_forest.fit(data[numerical_columns])

# Predict the outlier labels (-1 for outliers, 1 for inliers)
outlier_labels = isolation_forest.predict(data[numerical_columns])

# Filter the DataFrame to keep only the inliers
data = data[outlier_labels == 1]

#### Preprocessing

In [5]:
X = data.drop(columns=['math score'],axis=1)
y = data['math score']

In [6]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

#### Training

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [10]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.8517
- Mean Absolute Error: 0.2035
- R2 Score: 0.9967
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3858
- Mean Absolute Error: 0.1962
- R2 Score: 0.9994


Lasso
Model performance for Training set
- Root Mean Squared Error: 4.6138
- Mean Absolute Error: 3.6739
- R2 Score: 0.9029
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4.7860
- Mean Absolute Error: 3.8111
- R2 Score: 0.9056


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.8526
- Mean Absolute Error: 0.2268
- R2 Score: 0.9967
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.4024
- Mean Absolute Error: 0.2207
- R2 Score: 0.9993


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 3.3695
- Mean Absolute Error: 2.6404
- R2 Score: 0.9482
-----------------------

In [11]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.999386
2,Ridge,0.999332
7,CatBoosting Regressor,0.996360
6,XGBRegressor,0.995133
5,Random Forest Regressor,0.990849
4,Decision Tree,0.978810
3,K-Neighbors Regressor,0.925015
8,AdaBoost Regressor,0.918535
1,Lasso,0.905557
